In [1]:
from fluidics.FluidicsProgramBuilder import FluidicProgramBuilder
program_builder = FluidicProgramBuilder()


RuntimeError: The current Numpy installation ('C:\\Users\\dpshe\\miniconda3\\envs\\pycromanager\\lib\\site-packages\\numpy\\__init__.py') fails to pass a sanity check due to a bug in the windows runtime. See this issue for more information: https://tinyurl.com/y3dm3h86

In [ ]:
program_builder.define_program()